In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import tree
from sklearn import decomposition
from scipy.sparse import coo_matrix
from scipy.sparse import linalg

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['anime.csv', 'rating.csv']


In [2]:
ratings = pd.read_csv("../input/rating.csv")
ratings = ratings[ratings["user_id"] <= 40000]


print("Our columns are") 
print (list(ratings))
print("Num of observations is ", ratings.shape[0])
print("Num of unique anime series is ", len(ratings.anime_id.unique()))
print("Num of unique users is", len(ratings.user_id.unique()))

Our columns are
['user_id', 'anime_id', 'rating']
Num of observations is  4220266
Num of unique anime series is  9788
Num of unique users is 40000


We are going to be creating a model-based collaborative filtering recommender system. The model is going to train a decision tree for each item i(anime series) that decribes the rating of i as the dependent variable and all other items as independent variable, though we will reduce the dimensionality of these other items.

The first step is to shape our data.

In [3]:
#from scipy we can get a set up a sparse coordinate matrix with the ratings and users. 
R = coo_matrix((ratings['rating'].astype(float),(ratings['user_id'], ratings['anime_id'])))
print (R.shape)

#Convert to pandas dataframe 
R_p = pd.SparseDataFrame(R)



(40001, 34476)


To start, we'll just compute the decision tree for the first item, to illustrate what the algorithm is doing.

In [22]:
R_p = R_p.drop([R_p.columns[15000]], axis=1)
R_p

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,34436,34437,34438,34439,34440,34441,34442,34443,34444,34445,34446,34447,34448,34449,34450,34451,34452,34453,34454,34455,34456,34457,34458,34459,34460,34461,34462,34463,34464,34465,34466,34467,34468,34469,34470,34471,34472,34473,34474,34475
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,6.0,6.0,NaN,6.0,NaN,5.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#Remove i
Y = R_p.iloc[:,1]
R_new = R_p.drop([R_p.columns[1]], axis=1)
#Relabel rows 
R_new.index = range(len(R_new.index))
print(Y)

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
5         NaN
6         NaN
7         NaN
8         NaN
9         NaN
10        NaN
11        NaN
12        NaN
13       -1.0
14        NaN
15        NaN
16        NaN
17        NaN
18        NaN
19       10.0
20        NaN
21        9.0
22        NaN
23        9.0
24        NaN
25        NaN
26        NaN
27        NaN
28        NaN
29        NaN
         ... 
39971     NaN
39972     NaN
39973     9.0
39974     NaN
39975     NaN
39976     NaN
39977     NaN
39978     NaN
39979     NaN
39980     NaN
39981     NaN
39982     NaN
39983     NaN
39984     NaN
39985     NaN
39986     8.0
39987     NaN
39988     NaN
39989     NaN
39990     9.0
39991     NaN
39992     7.0
39993     NaN
39994     NaN
39995    10.0
39996     NaN
39997     8.0
39998     NaN
39999     8.0
40000    10.0
Name: 1, Length: 40001, dtype: float64
BlockIndex
Block locations: array([   13,    19,    21, ..., 39995, 39997, 39999], dtype=int32)
Block lengt

 Now we use this m x n-1 sparse dataframe to find an n-1 x n-1 covarience matrix.

In [6]:
#Make m x (n-1) into m x d dense matrix, which means first:
#Find n-1 x n-1 covarience matrix of m x (n-1) and find top-d eigenvectors for that matrix
R_new.index = range(len(R_new.index))

R_cov = R_new.T.cov()

W,P_d = linalg.eigs(R_cov,k=100,which='LM')
#print (R_cov)

KeyboardInterrupt: 

In [16]:
R_group = []
for row, col, dat in zip(R.row, R.col, R.data):
    R_group.append([row, col, dat])
R_group = pd.DataFrame(R_group,columns=['user_id','anime_id','rating']).groupby('user_id')
R_group.get_group(1)

,user_id,anime_id,rating
0,1,20,-1.0
1,1,24,-1.0
2,1,79,-1.0
3,1,226,-1.0
4,1,241,-1.0
5,1,355,-1.0
6,1,356,-1.0
7,1,442,-1.0
8,1,487,-1.0
9,1,846,-1.0


In [ ]:
R_final = []
u = 0
d = P_d.shape[1]

#Average contribution function
for u, I in R_group:
    for i in range(0,d):
        contribution = 0
        for a,row in I.iterrows():
            j = row['anime_id']
            contribution += row['rating']*W[j,i]
        ave_contribution = contribution/len(I)
        R_final[u,i] = group["anime_id"]
        
#Produces m x d matrix of average contribution a_ui for all u and i
    

In [24]:
Y_train = Y.dropna()

In [ ]:
dtree = tree.DecisionTreeClassifier()
dtree.fit(R_final,Y)
all_dtrees = []
all_dtree[0]=dtree

Repeat this process with all other items (preferably with a supercomputer).